# GAN(Generative Adversarial Network)
- 위조지폐범(생성자)과 경찰(구분자)가 한쪽은 최대한 속이려고 한쪽은 최대한 감별하려고 노력하는것과 비슷함
- 서로 대립(adversarial)하는 두 신경망을 경쟁시켜가며 결과물 생성 방법을 학습


## 구조
1. 실제 이미지를 주고 구분자(discriminator)에게 이 이미지가 진짜임을 판단하게 함
2. 생성자(Generator)를 통해 노이즈로부터 임의의 이미지를 만들고 이것을 다시 같은 구분자를 통해 진짜 이미지인지 판단
3. 이렇듯 생성자는 구분자를 속여 진짜처럼 보이게 하고, 구분자는 생성자가 만든 이미지를 최대한 가짜라고 구분하도록 훈련
4. 경쟁을 하면서 결과적으로 생성자는 실제 이미지와 상당히 비슷한 이미지 생성

## GAN 기본 모델 구현
- 숫자 생성

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
# 하이퍼 파라미터 설정
# 마지막의 noise는 생성자의 입력값으로 사용할 노이즈의 크기
total_epoch = 100
batch_size = 100
learning_rate = 0.0002
n_hidden = 256
n_input = 28 * 28
n_noise = 128

In [3]:
# 플레이스 홀더 설정
# 비지도 학습이므로 Y를 사용하지 않음
# 구분자에 넣을 이미지가 실제 이미지와 생성한 가짜 이미지 두 개이므로 Z변수 추가함
X = tf.placeholder(tf.float32,[None, n_input])
Z = tf.placeholder(tf.float32,[None, n_noise])

# 생성자 신경망에 사용할 변수 설정
# 첫 번째 가중치와 편향은 은닉층으로 출력하기 위한 변수들
# 두 번째 가중치와 편향은 출력층에 사용할 변수들
# 두번째 가중치의 변수 크기는 실제 이미지의 크기와 같아야함
# 그 크기는 n_input으로 28 x 28인 784가 됨
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev = 0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev = 0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))

# 구분자 신경망에 사용할 변수 설정
# 은닉층은 생성자와 동일하게 구성
# 구분자는 진짜와 얼마나 가까운가를 판단하는 값으로 0~1사이의 값을 출력
D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))
D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev=0.01))
D_b2 = tf.Variable(tf.zeros([1]))

# 실제 이미지를 판별하는 구분자 신경망과 생성한 이미지를 판별하는 구분자 신경망은 같은 변수를 사용해야 함
# 같은 신경망으로 구분을 시켜야 진짜 이미지와 가짜 이미지를 구분하는 특징들을 동시에 잡아낼 수 있기 때문

Instructions for updating:
Colocations handled automatically by placer.


### 생성자와 구분자 신경망 구성

In [4]:
# 생성자 신경망
# 무작위로 생성한 노이즈를 받아 가중치와 편향을 반영하여 은닉층을 만들고
# 은닉층에서 실제 이미지와 같은 크기의 결괏값을 출력하는 간단한 구성
def generator(noise_z):
    hidden = tf.nn.relu(tf.matmul(noise_z, G_W1) + G_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, G_W2) + G_b2)
    return output

# 구분자 신경망
# 0~1 사이의 스칼라값 하나를 출력
def discriminator(inputs):
    hidden = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, D_W2) + D_b2)
    return output

# 무작위한 노이즈를 만들어주는 함수 정의
def get_noise(batch_size, n_noise):
    return np.random.normal(size=(batch_size, n_noise))

In [5]:
# 노이즈 Z를 이용해 가짜이미지를 만들 생성자 G를 만들고
# 이 G가 만든 가짜 이미지와 진짜이미지 X를 구분자에 넣어 입력한 이미지가 진짜인지 판별
G = generator(Z)
D_gene = discriminator(G)
D_real = discriminator(X)

# GAN 모델의 최적화는 loss_G 와 loss_D 를 최대화 하는 것
# 진짜(D_real)는 1에 가까워 져야하고(진짜라고 판별)
# 가짜(D_gene)는 0에 가까워야 함(가짜라고 판별)
# D_real과 1에서 D_gene를 뺀 값을 더한 손실값으로 하여, 이 값을 최대화 하면 진짜판별기의 학습이 이뤄짐
loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1 - D_gene))
loss_G = tf.reduce_mean(tf.log(D_gene))

# loss_D 를 구할 때는 판별기 신경망에 사용되는 변수만 사용하고,
# loss_G 를 구할 때는 생성기 신경망에 사용되는 변수만 사용
D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]

# GAN 논문의 수식에 따르면 loss 를 극대화 해야하지만, minimize 하는 최적화 함수를 사용하기 때문에
# 최적화 하려는 loss_D 와 loss_G 에 음수 부호를 붙여줌
train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D, var_list=D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G, var_list=G_var_list)

In [6]:
# 신경망 모델 학습
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples/batch_size)
loss_val_D, loss_val_G = 0, 0

for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)

        # 판별기와 생성기 신경망을 각각 학습
        _, loss_val_D = sess.run([train_D, loss_D],
                                 feed_dict={X: batch_xs, Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G],
                                 feed_dict={Z: noise})

    print('Epoch:', '%04d' % epoch,
          'D loss: {:.4}'.format(loss_val_D),
          'G loss: {:.4}'.format(loss_val_G))

    # 학습이 되어가는 모습을 보기 위해 주기적으로 이미지를 생성하여 저장
    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict={Z: noise})

        fig, ax = plt.subplots(1, sample_size, figsize=(sample_size, 1))

        for i in range(sample_size):
            ax[i].set_axis_off()
            ax[i].imshow(np.reshape(samples[i], (28, 28)))

        plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
        plt.close(fig)
        
print('최적화 완료!')

Epoch: 0000 D loss: -0.5376 G loss: -1.958
Epoch: 0001 D loss: -0.4253 G loss: -2.095
Epoch: 0002 D loss: -0.2533 G loss: -2.29
Epoch: 0003 D loss: -0.4498 G loss: -1.508
Epoch: 0004 D loss: -0.3957 G loss: -1.859
Epoch: 0005 D loss: -0.3512 G loss: -2.364
Epoch: 0006 D loss: -0.2423 G loss: -2.541
Epoch: 0007 D loss: -0.249 G loss: -2.808
Epoch: 0008 D loss: -0.1321 G loss: -3.141
Epoch: 0009 D loss: -0.1639 G loss: -3.425
Epoch: 0010 D loss: -0.277 G loss: -2.693
Epoch: 0011 D loss: -0.3435 G loss: -2.662
Epoch: 0012 D loss: -0.4828 G loss: -2.05
Epoch: 0013 D loss: -0.5425 G loss: -2.432
Epoch: 0014 D loss: -0.3296 G loss: -2.527
Epoch: 0015 D loss: -0.4713 G loss: -1.996
Epoch: 0016 D loss: -0.3661 G loss: -2.463
Epoch: 0017 D loss: -0.4903 G loss: -2.506
Epoch: 0018 D loss: -0.329 G loss: -2.647
Epoch: 0019 D loss: -0.3301 G loss: -2.747
Epoch: 0020 D loss: -0.4499 G loss: -2.633
Epoch: 0021 D loss: -0.415 G loss: -2.592
Epoch: 0022 D loss: -0.3673 G loss: -2.287
Epoch: 0023 D los

## 원하는 숫자 생성

In [8]:
total_epoch = 100
batch_size = 100
learning_rate = 0.0002
n_hidden = 256
n_input = 28 * 28
n_noise = 128
n_class = 10

In [9]:
X = tf.placeholder(tf.float32, [None, n_input])
# 노이즈와 실제 이미지에, 그에 해당하는 숫자에 대한 정보를 넣어주기 위해 사용
Y = tf.placeholder(tf.float32, [None, n_class])
Z = tf.placeholder(tf.float32, [None, n_noise])

In [10]:
def generator(noise, labels):
    with tf.variable_scope('generator'):
        # noise 값에 labels 정보를 추가
        inputs = tf.concat([noise, labels], 1)
        # tf.layers.dense 함수를 이용해 은닉층(hidden) 작성
        hidden = tf.layers.dense(inputs, n_hidden, activation=tf.nn.relu)
        # 진짜 이미지와 같은 크기의 값을 만드는 출력층 작성
        output = tf.layers.dense(hidden, n_input, activation=tf.nn.sigmoid)

    return output

# 구분자 신경망 정의
def discriminator(inputs, labels, reuse=None):
    with tf.variable_scope('discriminator') as scope:
        # 노이즈에서 생성한 이미지와 실제 이미지를 판별하는 모델의 변수를 동일하게 하기 위해,
        # 이전에 사용되었던 변수를 재사용
        if reuse:
            scope.reuse_variables()

        inputs = tf.concat([inputs, labels], 1)
        hidden = tf.layers.dense(inputs, n_hidden, activation=tf.nn.relu)
        output = tf.layers.dense(hidden, 1, activation=None)

    return output

def get_noise(batch_size, n_noise):
    return np.random.uniform(-1., 1., size=[batch_size, n_noise])

In [11]:
# 생성자 구성
# 생성자에는 레이블 정보를 추가하여 추후 레이블 정보에 해당하는 이미지를 생성할 수 있도록 유도
G = generator(Z, Y)
D_real = discriminator(X, Y)
D_gene = discriminator(G, Y, True)

# D_real 값은 1에, D_gene값은 0에 가까워 지게 하는 것인데
# tf.nn.sigmoid_cross_entropy_with_logits 함수를 이요하면 보다 간편하게 작성가능
loss_D_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real, labels=tf.ones_like(D_real)))
loss_D_gene = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_gene, labels=tf.zeros_like(D_gene)))
# loss_D_real 과 loss_D_gene 을 더한 뒤 이 값을 최소화하면 구분자를 학습시킬 수 있음
loss_D = loss_D_real + loss_D_gene
# 가짜 이미지를 진짜에 가깝게 만들도록 생성망을 학습시키기 위해, D_gene 을 최대한 1에 가깝도록 만드는 손실함수
loss_G = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_gene, labels=tf.ones_like(D_gene)))

# TensorFlow 에서 제공하는 유틸리티 함수를 이용해
# discriminator 와 generator scope 에서 사용된 변수들을 쉽게 가져올 수 있음
vars_D = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='discriminator')
vars_G = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='generator')

train_D = tf.train.AdamOptimizer().minimize(loss_D, var_list=vars_D)
train_G = tf.train.AdamOptimizer().minimize(loss_G, var_list=vars_G)

Instructions for updating:
Use keras.layers.dense instead.


In [13]:
# 신경망 모델 학습
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples/batch_size)
loss_val_D, loss_val_G = 0, 0

for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)

        _, loss_val_D = sess.run([train_D, loss_D],
                                 feed_dict={X: batch_xs, Y: batch_ys, Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G],
                                 feed_dict={Y: batch_ys, Z: noise})

    print('Epoch:', '%04d' % epoch,
          'D loss: {:.4}'.format(loss_val_D),
          'G loss: {:.4}'.format(loss_val_G))

    # 학습이 되어가는 모습을 보기 위해 주기적으로 레이블에 따른 이미지를 생성하여 저장
    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G,
                           feed_dict={Y: mnist.test.labels[:sample_size],
                                      Z: noise})

        fig, ax = plt.subplots(2, sample_size, figsize=(sample_size, 2))

        for i in range(sample_size):
            ax[0][i].set_axis_off()
            ax[1][i].set_axis_off()

            ax[0][i].imshow(np.reshape(mnist.test.images[i], (28, 28)))
            ax[1][i].imshow(np.reshape(samples[i], (28, 28)))

        plt.savefig('samples2/{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
        plt.close(fig)

print('최적화 완료!')

Epoch: 0000 D loss: 0.003557 G loss: 7.655
Epoch: 0001 D loss: 0.02505 G loss: 6.854
Epoch: 0002 D loss: 0.01085 G loss: 8.782
Epoch: 0003 D loss: 0.01865 G loss: 8.482
Epoch: 0004 D loss: 0.04829 G loss: 7.966
Epoch: 0005 D loss: 0.02063 G loss: 7.595
Epoch: 0006 D loss: 0.006697 G loss: 8.0
Epoch: 0007 D loss: 0.1096 G loss: 7.643
Epoch: 0008 D loss: 0.1533 G loss: 6.908
Epoch: 0009 D loss: 0.114 G loss: 7.381
Epoch: 0010 D loss: 0.1715 G loss: 5.471
Epoch: 0011 D loss: 0.2219 G loss: 4.871
Epoch: 0012 D loss: 0.3874 G loss: 4.96
Epoch: 0013 D loss: 0.2477 G loss: 4.651
Epoch: 0014 D loss: 0.3568 G loss: 4.587
Epoch: 0015 D loss: 0.4408 G loss: 3.941
Epoch: 0016 D loss: 0.4865 G loss: 3.585
Epoch: 0017 D loss: 0.292 G loss: 3.82
Epoch: 0018 D loss: 0.3567 G loss: 4.433
Epoch: 0019 D loss: 0.5217 G loss: 3.196
Epoch: 0020 D loss: 0.7289 G loss: 2.802
Epoch: 0021 D loss: 0.8913 G loss: 2.158
Epoch: 0022 D loss: 0.5318 G loss: 3.297
Epoch: 0023 D loss: 0.6443 G loss: 2.216
Epoch: 0024 D

- 초짜 대학원생 입장에서 이해하는 GAN
  https://goo.gl/ZvSvtm 참조